## AI model to predict waste images. 
The dataset is acquired from 
https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification/data


In [5]:

import os 
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torchvision.transforms.v2 as v2
import torch
from torch import nn, optim

### Parameters

In [2]:
dataset_folder  = "Dataset/images"
batch_size = 32
train_ratio = 0.7
valid_ratio = 0.15
test_ratio = 0.15
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

print (f"Settings ... \n dataset path \t\t {dataset_folder} \n batch_size \t\t {batch_size}" +
       f"\n train_ratio \t\t {train_ratio} \n validation ratio \t {valid_ratio} \n test ratio \t\t {test_ratio} \n device \t\t {device}")

Using device: cpu
Settings ... 
 dataset path 		 Dataset/images 
 batch_size 		 32
 train_ratio 		 0.7 
 validation ratio 	 0.15 
 test ratio 		 0.15 
 device 		 cpu


In [6]:
class_labels = [name for name in os.listdir(dataset_folder) if not name.startswith(".DS_Store")]
print(class_labels)
num_classes = len(class_labels)
print(num_classes)

['disposable_plastic_cutlery', 'food_waste', 'office_paper', 'glass_food_jars', 'aluminum_soda_cans', 'magazines', 'clothing', 'plastic_shopping_bags', 'plastic_soda_bottles', 'styrofoam_food_containers', 'aerosol_cans', 'aluminum_food_cans', 'newspaper', 'eggshells', 'glass_cosmetic_containers', 'paper_cups', 'plastic_water_bottles', 'coffee_grounds', 'steel_food_cans', 'plastic_cup_lids', 'cardboard_packaging', 'cardboard_boxes', 'plastic_straws', 'styrofoam_cups', 'glass_beverage_bottles', 'shoes', 'plastic_trash_bags', 'tea_bags', 'plastic_food_containers', 'plastic_detergent_bottles']
30


In [7]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transformations = v2.Compose([
    v2.ToImage(),
    v2.RandomHorizontalFlip(),
    v2.RandomVerticalFlip(),
    v2.RandomResizedCrop(size=(256, 256), antialias=True),
    v2.RandomRotation(degrees = (0, 170)),
    v2.ToDtype(dtype = torch.float32, scale = True),
    v2.Resize(size = (256, 256), antialias = True),
    v2.Normalize(mean, std)
])

dataset = ImageFolder(dataset_folder, transform = transformations)

### Creating and splitting the dataset

In [8]:
train_size = int(len(dataset) * train_ratio)
valid_size = int(len(dataset) * valid_ratio)
test_szie = int(len(dataset) * test_ratio)

train_dataset, valid_dataset, test_dataset = random_split(dataset,[train_size, valid_size, test_szie])

train_dataloader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
valid_dataloader = DataLoader(valid_dataset,shuffle=True,batch_size=batch_size)
test_dataloader = DataLoader(test_dataset,shuffle=True, batch_size=batch_size)

print(f"Lenght of training dataset {len(train_dataloader.dataset)}")
print(f"Lenght of validation dataset {len(valid_dataloader.dataset)}")
print(f"Lenght of test dataset {len(test_dataloader.dataset)}")


Lenght of training dataset 10500
Lenght of validation dataset 2250
Lenght of test dataset 2250


## CNN model 

In [9]:
class CNN(nn.Module):
    def __init__ (self):
        super().__init__()
        self.network = nn.Sequential(

            nn.Conv2d(in_channels = 3, out_channels = 4, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
                                                                                                    #256,256,4
            nn.Conv2d(in_channels = 4, out_channels = 8, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
                                                                                                    #256,256,8
            nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #128,128,16
            nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #64,64,32
            nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #32,32,64
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #16,16,64
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #8,8,64
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
                                                                                                    #8,8,128
            nn.Flatten(),
            nn.Linear(128*8*8, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
            
        )


    def forward(self, x):
            return self.network(x)

In [10]:
model = CNN()
model = model.to(device)    #converting model to gpu from cpu
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3 )
print("Compiled Successfully")

Compiled Successfully


In [ ]:
def calculate_accuracy(loader, model):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x,y = x.to(device), y.to(device)
            scores = model(x)
            _, predictions = scores.max(1)
            correct += (predictions == y).sum().item()
            total += predictions.size(0)
    model.train()
    return correct/total

In [11]:

import time
for epoch in range(20):
    start_time = time.time()
    for x,y in train_dataloader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss = criteria(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    end_time = time.time()
    epoch_time = end_time - start_time
    acc = calculate_accuracy(train_dataloader, model)*100
    print(f"loss value of epoch {epoch+1} = {loss} in {epoch_time:.4f} seconds, train accuracy {acc:.2f}")

loss value of epoch 1 = 3.417145252227783 in 958.8621 seconds
loss value of epoch 2 = 2.8865630626678467 in 1410.2358 seconds
loss value of epoch 3 = 3.0759453773498535 in 1011.2184 seconds


KeyboardInterrupt: 

In [ ]:

print(f"Training Accuracy is {calculate_accuracy(train_dataloader, model)*100}")

print(f"Testing Accuracy is {calculate_accuracy(test_dataloader, model)*100}")